# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [3]:
#main.py file
import pandas as pd

#url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv"
url2 = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv"
url3 = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv"
previous = "output_3lab.csv"

insurance_df_previous = pd.read_csv(previous)
#insurance_df = pd.read_csv(url)
insurance_df2 = pd.read_csv(url2)
insurance_df3 = pd.read_csv(url3)

from my_data_cleaning_functions_3lab_vf import adjust_columns, inconsistencies, df_replace_education, formating_data_types, dealing_duplicates, save_to_csv, df_merge

insurance_df2 = adjust_columns(insurance_df2,2)
insurance_df3 = adjust_columns(insurance_df3,3)

insurance_df2 = inconsistencies(insurance_df2)
insurance_df3 = inconsistencies(insurance_df3)

insurance_df2 = df_replace_education(insurance_df2)
insurance_df3 = df_replace_education(insurance_df3)

insurance_df2 = formating_data_types(insurance_df2)
insurance_df3 = formating_data_types(insurance_df3)

insurance_df2 = dealing_duplicates(insurance_df2)
insurance_df3 = dealing_duplicates(insurance_df3)

save_to_csv(insurance_df2, 2)
save_to_csv(insurance_df3, 3)

df_vfinal = df_merge(insurance_df_previous, insurance_df2, insurance_df3)


#my_data_cleaning_functions_3lab_vf.py file
import pandas as pd
    
def adjust_columns(insurance_data, num):
    insurance_data.columns = insurance_data.columns.str.lower() #- *Column names should be in lower case*
    insurance_data.columns = insurance_data.columns.str.replace(' ','_') #- *White spaces in column names replaced by `_`
    insurance_data.rename(columns = {'st':'state'},inplace=True) 
    return insurance_data
    
def inconsistencies(insurance_data):
    #first step to deal with inconsistencies is to drop the NAN values
    insurance_data.dropna(inplace=True)
    
    #1.1 - Gender column contains various inconsistent values such as "F", "M", "Femal", "Male", "female", which need to be standardized, for example, to "M" and "F".*
    insurance_data['gender'] = insurance_data['gender'].replace({"Femal":"F", "female":"F","Male":"M"})
    
    #1.2 - State abbreviations can be replaced with its full name, for example "AZ": "Arizona", "Cali": "California", "WA": "Washington"*
    states_list = {'AZ':'Arizona', 'Cali':'California', 'WA':'Washington'}
    insurance_data['state'] = insurance_data['state'].replace(states_list)
    
    return insurance_data

def df_replace_education(insurance_data):
    #1.3 - In education, "Bachelors" could be replaced by "Bachelor"*
    #insurance_data['education'].replace({'Bachelors':'Bachelor'},inplace=True)
    insurance_data['education'] = insurance_data['education'].str.replace('Bachelors','Bachelor')
    
    #1.4 - In Customer Lifetime Value, delete the `%` character*
    if insurance_data['customer_lifetime_value'].dtype == float:
        insurance_data['customer_lifetime_value'] = insurance_data['customer_lifetime_value'].astype(str)
        insurance_data['customer_lifetime_value'] = insurance_data['customer_lifetime_value'].str.replace('%','')
        insurance_data['customer_lifetime_value'] = insurance_data['customer_lifetime_value'].astype(float)
    else:
        insurance_data['customer_lifetime_value'] = insurance_data['customer_lifetime_value'].str.replace('%','')
        insurance_data['customer_lifetime_value'] = insurance_data['customer_lifetime_value'].astype(float)

    #1.5 - In vehicle class, "Sports Car", "Luxury SUV" and "Luxury Car" could be replaced by "Luxury"*
    vehicle_list = {"Sports Car":"Luxury", "Luxury SUV":"Luxury","Luxury Car":"Luxury"}
    insurance_data['vehicle_class'] = insurance_data['vehicle_class'].replace(vehicle_list)
    
    return insurance_data

def formating_data_types(insurance_data):
    #changing 'number_of_open_complaints' to int (numeric)
    if insurance_data['number_of_open_complaints'].any != "na" or insurance_data['number_of_open_complaints'].any != "NA":
        insurance_data['number_of_open_complaints'] = insurance_data['number_of_open_complaints'].astype(str)#
        insurance_data['number_of_open_complaints'] = insurance_data['number_of_open_complaints'].str.split('/').str[1].astype(float)
        
    return insurance_data

def dealing_duplicates(insurance_data):
    insurance_data.drop_duplicates(keep='first', inplace=True)
    insurance_data.reset_index()
    return insurance_data
 
def save_to_csv(insurance_data, num):
    insurance_data.to_csv(f'output_df{num}.csv')   

def df_merge(data1, data2, data3):
    #print(f"\nIs{data1} none?", data1 is None)
    #print(f"\nIs{data2} none?", data2 is None)
    #print(f"Data 1:\n{data1.columns}")
    #print(f"Data 2:\n{data2.columns}")
    
    df_final = pd.merge(data1, data2, on='customer', how='right')
    df_final2 = pd.merge(df_final, data3, on='customer', how='right')
   

    print(f"Previous exercise shape:{data1.shape}")
    print(f"Data 2 shape:{data2.shape}")
    print(f"Df final shape:{df_final.shape}")
    
    print(f"Data 3 shape:{data3.shape}")
    print(df_final2.shape)

    print(f"Merged dataframe\n {df_final2.head()}")

Previous exercise shape:(952, 12)
Data 2 shape:(988, 11)
Df final shape:(988, 22)
Data 3 shape:(7070, 11)
(7070, 32)
Merged dataframe
    Unnamed: 0 customer state_x gender_x education_x  \
0         NaN  SA25987     NaN      NaN         NaN   
1         NaN  TB86706     NaN      NaN         NaN   
2         NaN  ZL73902     NaN      NaN         NaN   
3         NaN  KX23516     NaN      NaN         NaN   
4         NaN  FN77294     NaN      NaN         NaN   

   customer_lifetime_value_x  income_x  monthly_premium_auto_x  \
0                        NaN       NaN                     NaN   
1                        NaN       NaN                     NaN   
2                        NaN       NaN                     NaN   
3                        NaN       NaN                     NaN   
4                        NaN       NaN                     NaN   

   number_of_open_complaints_x policy_type_x  ...       state  \
0                          NaN           NaN  ...  Washington   
1      

# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [6]:
#Data loading
import pandas as pd
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv"
marketing_df = pd.read_csv(url)

In [7]:
#Data understanding
display(marketing_df.head())
display(marketing_df.dtypes)
print(f"Shape: {marketing_df.shape}")
#Check the missing values for each column in the dataframe
marketing_df.isnull().sum()
#checl for duplicated values
marketing_df.duplicated().sum()


,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,A,2
1,1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,...,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,A,1
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,...,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,2
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,1
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,...,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,A,1


unnamed:_0                         int64
customer                          object
state                             object
customer_lifetime_value          float64
response                          object
coverage                          object
education                         object
effective_to_date                 object
employmentstatus                  object
gender                            object
income                             int64
location_code                     object
marital_status                    object
monthly_premium_auto               int64
months_since_last_claim          float64
months_since_policy_inception      int64
number_of_open_complaints        float64
number_of_policies                 int64
policy_type                       object
policy                            object
renew_offer_type                  object
sales_channel                     object
total_claim_amount               float64
vehicle_class                     object
vehicle_size    

Shape: (10910, 27)


0

In [8]:
#Data cleaning:
#No need to use the dropna() or fillna() because we don't have nan values in the dataframe;
#No need to drop_duplicates() because there are no duplicated values - marketing_df.dropna(inplace=True);
#No need to standardize column names

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

In [26]:
#Which sales channel brought the most sales in terms of total revenue.
#marketing_df['income']
#marketing_df['sales_channel']
#marketing_df['total_claim_amount']

revenue = marketing_df.groupby('sales_channel')['total_claim_amount'].sum().reset_index()
display(revenue)
print(f"The channel who brought the most sales was sales channel {revenue['sales_channel'][0]}")

,sales_channel,total_claim_amount
0,Agent,1.810227e+06
1,Branch,1.301204e+06
2,Call Center,9.266008e+05
3,Web,7.066000e+05


The channel who brought the most sales was sales channel Agent


In [30]:
#Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail)
total_revenue = pd.pivot_table(
    marketing_df, 
    values='total_claim_amount', 
    index=marketing_df['sales_channel'], 
    aggfunc='sum').round(2)

display(total_revenue)

,total_claim_amount
sales_channel,
Agent,1810226.82
Branch,1301204.00
Call Center,926600.82
Web,706600.04


2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [68]:
clv_average = marketing_df['customer_lifetime_value'].mean()

clv_average = pd.pivot_table(
    marketing_df,
    values='customer_lifetime_value',
    index = marketing_df['gender'], columns=marketing_df['education'],
    aggfunc='mean')

display(clv_average)


education,Bachelor,College,Doctor,High School or Below,Master
gender,,,,,
F,7874.269478,7748.823325,7328.508916,8675.220201,8157.053154
M,7703.601675,8052.459288,7415.333638,8149.687783,8168.832659


## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [ ]:
# Your code goes here